<a href="https://colab.research.google.com/github/FernandoBRdgz/inteligencia_artificial/blob/main/algoritmos_gen%C3%A9ticos/optimizaci%C3%B3n_combinatoria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introducción

Un algoritmo genético es una técnica de optimización que se inspira en la evolución biológica para encontrar soluciones a problemas complejos. En términos generales, consiste en crear una población inicial de soluciones candidatas, aplicar operaciones de selección, cruza y mutación para generar nuevas soluciones, y evaluar el rendimiento de cada solución en base a una función objetivo definida. Las soluciones con mejores rendimientos tienen más posibilidades de sobrevivir y reproducirse en la siguiente generación, mientras que las soluciones peor adaptadas son eliminadas.

La idea detrás de los algoritmos genéticos es que, al aplicar una selección natural sobre soluciones candidatas, se pueden encontrar soluciones mejores y mejores con el tiempo. En la inteligencia artificial, los algoritmos genéticos se utilizan a menudo en problemas de optimización, como encontrar la mejor solución para un problema de programación lineal o de asignación de recursos. También se pueden utilizar en la generación de soluciones creativas en áreas como la arquitectura o el diseño.

**Objetivo** En este ejemplo, veremos como implementar el problema de la mochila, también conocido como problema de la mochila 0-1 o problema Knapsack. Es un problema de optimización combinatoria. El problema consiste en seleccionar un subconjunto de elementos de un conjunto dado, de tal manera que la suma de sus pesos no exceda una capacidad dada, y que la suma de sus valores sea lo más grande posible.

El problema se puede formular de la siguiente manera: se tiene una mochila con una capacidad máxima de peso W, y se tiene un conjunto de n elementos, cada uno con un valor v y un peso w. El objetivo es seleccionar un subconjunto de elementos cuya suma de pesos no exceda W, y cuya suma de valores sea lo más grande posible.

El problema de la mochila es un problema NP-completo, lo que significa que no se conoce un algoritmo eficiente para resolverlo en el peor de los casos. Sin embargo, existen varios algoritmos heurísticos y metaheurísticos que pueden proporcionar soluciones aproximadas al problema de manera eficiente en la mayoría de los casos.

In [1]:
from functools import partial
from collections import namedtuple
from random import choices, randint, randrange, random

In [2]:
# Función para generar un genoma aleatorio de longitud "length"
def generate_genome(length):
    # Se utiliza la función "choices" del módulo "random" para generar un genoma aleatorio
    return choices([0, 1], k=length)

In [3]:
# Función para generar una población aleatoria de tamaño "size", donde cada individuo tiene un genoma de longitud "genome_length"
def generate_population(size, genome_length):
    # Se utiliza la función "generate_genome" para generar cada individuo de la población
    return [generate_genome(genome_length) for _ in range(size)]

In [4]:
# Función para realizar un cruce de un punto entre dos genomas "a" y "b"
def single_point_crossover(a, b):
    # Se verifica que los genomas "a" y "b" tengan la misma longitud
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")
    # Se obtiene la longitud de los genomas
    length = len(a)
    # Si la longitud es menor a 2, se devuelve los mismos genomas
    if length < 2:
        return a, b
    # Se elige un punto de cruce aleatorio entre 1 y la longitud del genoma menos 1
    p = randint(1, length - 1)
    # Se intercambian las partes del genoma a partir del punto de cruce para generar dos nuevos genomas
    return a[0:p] + b[p:], b[0:p] + a[p:]

In [5]:
# Función para mutar un genoma. "num" indica la cantidad de mutaciones a realizar, y "probability" es la probabilidad de que un bit sea invertido.
def mutation(genome, num=1, probability=0.5):
    # Se realizan "num" mutaciones en el genoma
    for _ in range(num):
        # Se elige un índice aleatorio del genoma
        index = randrange(len(genome))
        # Se invierte el bit en el índice elegido con una probabilidad "probability"
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
    # Se devuelve el genoma mutado
    return genome

In [6]:
# Función para calcular el fitness total de una población
def population_fitness(population, fitness_func):
    # Se utiliza la función "sum" para calcular la suma de los fitness de cada individuo de la población
    return sum([fitness_func(genome) for genome in population])

In [7]:
# Función para seleccionar una pareja de individuos para el cruce
def selection_pair(population, fitness_func):
    # Se utiliza la función "choices" del módulo "random" para elegir dos individuos de la población con una probabilidad ponderada por su fitness
    return choices(population=population, weights=[fitness_func(gene) for gene in population], k=2)

In [8]:
# Función para ordenar una población de mayor a menor fitness
def sort_population(population, fitness_func):
    # Se utiliza la función "sorted" para ordenar la población en función de su fitness, de mayor a menor
    return sorted(population, key=fitness_func, reverse=True)

In [9]:
# Función para convertir un genoma a una cadena de caracteres
def genome_to_string(genome):
    # Se utiliza la función "join" para concatenar los elementos del genoma como una cadena de caracteres
    return "".join(map(str, genome))

In [10]:
def print_stats(population, generation_id, fitness_func):
    print("Generación %02d" % generation_id)
    print("=============")
    print("Población: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
    print("Ajuste medio: %f" % (population_fitness(population, fitness_func) / len(population)))
    sorted_population = sort_population(population, fitness_func)
    print("Mejor: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
    print("Peor: %s (%f)" % (genome_to_string(sorted_population[-1]), fitness_func(sorted_population[-1])))
    print("")
    return sorted_population[0]

In [11]:
# Función para ejecutar el algoritmo evolutivo
def run_evolution(populate_func, fitness_func, fitness_limit, selection_func=selection_pair, crossover_func=single_point_crossover,
                  mutation_func=mutation, generation_limit=100, printer=print_stats):
    
    # Se crea la población inicial
    population = populate_func()

    # Se ejecutan las generaciones del algoritmo
    for i in range(generation_limit):
        # Se ordena la población en función de su fitness
        population = sorted(population, key=lambda genome: fitness_func(genome), reverse=True)

        # Se imprime la información de la población si se proporciona una función de impresión
        if printer is not None:
            printer(population, i, fitness_func)

        # Si se alcanza el fitness límite, se sale del bucle
        if fitness_func(population[0]) >= fitness_limit:
            break

        # Se seleccionan los dos individuos con mayor fitness para la siguiente generación
        next_generation = population[0:2]

        # Se generan los descendientes a partir de los padres seleccionados mediante cruce y mutación
        for j in range(int(len(population) / 2) - 1):
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        # Se actualiza la población para la siguiente generación
        population = next_generation

    # Se devuelve la población final y el número de generaciones ejecutadas
    return population, i

In [12]:
Thing = namedtuple('Thing', ['name', 'value', 'weight'])

In [13]:
def generate_things(num):
    # Genera una lista de objetos "Thing" con valores y pesos aleatorios.
    return [Thing(f"thing{i}", i, i) for i in range(1, num+1)]

In [14]:
def fitness(genome, things, weight_limit):
    # Verifica que el genoma y la lista de objetos tengan la misma longitud
    if len(genome) != len(things):
        raise ValueError("el genoma y los objetos deben ser de la misma longitud")

    # Inicializa el peso y el valor de la mochila en cero
    weight = 0
    value = 0

    # Recorre la lista de objetos y agrega el peso y el valor de los objetos incluidos en la mochila
    for i, thing in enumerate(things):
        if genome[i] == 1:
            weight += thing.weight
            value += thing.value

            # Si el peso de la mochila supera el límite permitido, retorna un valor de cero
            if weight > weight_limit:
                return 0

    # Si el peso de la mochila está dentro del límite permitido, retorna el valor total de los objetos incluidos
    return value

In [15]:
# Primer ejemplo con 5 objetos
first_example = [
    Thing('Laptop', 500, 2200),
    Thing('Audífonos', 150, 160),
    Thing('Taza de café', 60, 350),
    Thing('Cuaderno', 40, 333),
    Thing('Botella de agua', 30, 192),
]
# Segundo ejemplo con 10 objetos
second_example = first_example + [
    Thing('Dulces', 5, 25),
    Thing('Calcetines', 10, 38),
    Thing('Pañuelos', 15, 80),
    Thing('Celular', 500, 200),
    Thing('Batería', 100, 70)
]

In [16]:
# things = generate_things(22)
things = second_example
weight_limit = 3000

print("Peso límite: %dkg" % weight_limit)

Peso límite: 3000kg


In [17]:
def bruteforce(things, weight_limit: int):
    # Verifica si la lista de objetos está vacía
    if len(things) == 0:
        return 0, []

    # Inicializa el valor máximo y la lista de objetos incluidos en la mochila que tienen el valor máximo
    max_value = 0
    max_valued_packed = []

    # Recorre la lista de objetos
    for i, thing in enumerate(things):
        # Si el peso del objeto es mayor que el límite permitido, pasa al siguiente objeto
        if thing.weight > weight_limit:
            continue

        # Llama recursivamente a la función bruteforce con el resto de los objetos y el peso limitado restante
        value, packed = bruteforce(things[i + 1:], weight_limit - thing.weight)

        # Si el valor del objeto actual más el valor de los objetos incluidos en la mochila recursiva es mayor o igual que el valor máximo,
        # actualiza el valor máximo y la lista de objetos incluidos en la mochila con el valor actual y los objetos actuales más los objetos incluidos
        # en la mochila recursiva
        if value + thing.value >= max_value:
            max_value = value + thing.value
            max_valued_packed = [thing] + packed

    # Retorna el valor máximo y la lista de objetos incluidos en la mochila que tienen el valor máximo
    return max_value, max_valued_packed

In [18]:
# Resolución por fuerza bruta
result = bruteforce(things, weight_limit)
result

(1310,
 [Thing(name='Laptop', value=500, weight=2200),
  Thing(name='Audífonos', value=150, weight=160),
  Thing(name='Botella de agua', value=30, weight=192),
  Thing(name='Dulces', value=5, weight=25),
  Thing(name='Calcetines', value=10, weight=38),
  Thing(name='Pañuelos', value=15, weight=80),
  Thing(name='Celular', value=500, weight=200),
  Thing(name='Batería', value=100, weight=70)])

In [19]:
# Resolución por algoritmo genético
population, generations = run_evolution(
		populate_func=partial(generate_population, size=10, genome_length=len(things)),
		fitness_func=partial(fitness, things=things, weight_limit=weight_limit),
		fitness_limit=result[0],
		generation_limit=100)

Generación 00
Población: [1001001010, 0101000111, 1100110101, 0010001111, 1100000100, 1001010100, 0101011101, 0110110000, 0011101000, 1011011010]
Ajuste medio: 527.000000
Mejor: 1001001010 (1050.000000)
Peor: 1011011010 (0.000000)

Generación 01
Población: [1100000111, 1001001010, 0101011111, 1100111101, 0101000111, 1100110101, 0101011101, 0100110101, 0010001100, 1110011111]
Ajuste medio: 625.500000
Mejor: 1100000111 (1265.000000)
Peor: 1110011111 (0.000000)

Generación 02
Población: [1100000111, 1100110010, 1001001010, 1001001010, 0101100111, 0101011111, 0101000110, 1001001101, 0100111101, 1101000111]
Ajuste medio: 788.500000
Mejor: 1100000111 (1265.000000)
Peor: 1101000111 (0.000000)

Generación 03
Población: [1100000111, 1100110010, 1100110010, 1100000110, 1001101010, 0101111111, 0101100111, 0101011111, 0101001111, 0100000111]
Ajuste medio: 996.500000
Mejor: 1100000111 (1265.000000)
Peor: 0100000111 (765.000000)

Generación 04
Población: [1100110011, 1100000111, 1100000111, 11001100

In [20]:
def from_genome(genome, things):
    # Inicializa una lista vacía para almacenar los objetos incluidos en la mochila
    result = []
    
    # Recorre la lista de objetos y agrega los objetos que se incluyen en la mochila a la lista resultante
    for i, thing in enumerate(things):
        if genome[i] == 1:
            result += [thing]

    # Retorna la lista de objetos incluidos en la mochila
    return result

In [21]:
def to_string(things):
    # Crea una cadena que representa la lista de objetos, separados por comas y entre corchetes
    return f"[{', '.join([t.name for t in things])}]"

In [22]:
def value(things):
    # Retorna la suma de los valores de los objetos en la lista de objetos
    return sum([t.value for t in things])

In [23]:
def weight(things):
    # Retorna la suma de los pesos de los objetos en la lista de objetos
    return sum([p.weight for p in things])

In [24]:
def print_stats2(things):
    print(f"Things: {to_string(things)}")
    print(f"Value {value(things)}")
    print(f"Weight: {weight(things)}")

In [25]:
# Resultado por algoritmo genético
sack = from_genome(population[0], things)
print_stats2(sack)

Things: [Laptop, Audífonos, Botella de agua, Dulces, Calcetines, Pañuelos, Celular, Batería]
Value 1310
Weight: 2965


In [26]:
# Resultado por fuerza bruta
result

(1310,
 [Thing(name='Laptop', value=500, weight=2200),
  Thing(name='Audífonos', value=150, weight=160),
  Thing(name='Botella de agua', value=30, weight=192),
  Thing(name='Dulces', value=5, weight=25),
  Thing(name='Calcetines', value=10, weight=38),
  Thing(name='Pañuelos', value=15, weight=80),
  Thing(name='Celular', value=500, weight=200),
  Thing(name='Batería', value=100, weight=70)])

**Referencias**

* https://arpitbhayani.me/blogs/genetic-knapsack
* https://plainenglish.io/blog/genetic-algorithm-in-python-101-da1687d3339b
* https://www.kdnuggets.com/2023/01/knapsack-problem-genetic-programming-python.html